In [317]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from geopy.distance import geodesic
from shapely.geometry import Point, LineString

In [318]:
# Read the data from the csv file
airport_info = pd.read_csv('Assignment_Data/Group_4_Airport_info.csv')

# We make a dictionary with the airport code as the key the rest of the columns as sub-dictionaries
airport_dict = {}

for index, row in airport_info.iterrows():
    airport_dict[row['ICAO Code']] = row.to_dict()

In [319]:
# Create distance matrix
# Create a list of all the airports
airport_list = airport_info['ICAO Code'].tolist()

# Create a dictionary of all the airports and their coordinates
airport_coords = {}

for i in airport_list:
    airport_coords[i] = (airport_info[airport_info['ICAO Code'] == i]['Latitude (deg)'].values[0],
                       airport_info[airport_info['ICAO Code'] == i]['Longitude (deg)'].values[0])

# Create a distance matrix
distance_dict = {}

# Fill the distance matrix in a dict with the distances between the airports
for i in airport_list:
    distance_dict[i] = {}
    for j in airport_list:
        distance_dict[i][j] = geodesic(airport_coords[i], airport_coords[j]).km

In [320]:
# Import aircraft info
aircraft_info = pd.read_csv('Assignment_Data/Aircraft_info.csv')

# Add a column for BT (Block Time) to the aircraft_info dataframe
aircraft_info['BT'] = 10  # [hours]

# Create a dictionary with the aircraft types as keys and the rest of the columns as sub-dictionaries
aircraft_dict = {}
for index, row in aircraft_info.iterrows():
    aircraft_dict[row['AC_type']] = row.to_dict()

# Rename the main keys in the aircraft dictionary to a numeric value with format AC_#

# Create a list of the keys in the aircraft dictionary
keys = list(aircraft_dict.keys())

# Create a new dictionary with the new keys
new_keys = {}
for i in range(len(keys)):
    new_keys[keys[i]] = 'AC_' + str(i+1)

aircraft_types = list(new_keys.values())

# Create a new dictionary with the new keys
aircraft_dict_new = {}
for i in range(len(keys)):
    aircraft_dict_new['AC_' + str(i+1)] = aircraft_dict[keys[i]]

# Replace the keys in the aircraft dictionary with the new keys
aircraft_dict = aircraft_dict_new


In [420]:
# Import demand info
demand_info = pd.read_csv('Assignment_Data/Group_4_Demand.csv')
demand_info.rename(columns={'Unnamed: 0': 'Origin'}, inplace=True)

# Create a dictionary with the demand info
demand_dict = {}
for index, row in demand_info.iterrows():
    demand_dict[row['Origin']] = row.to_dict()

# Remove the origin column from the demand_dict
for i in demand_dict:
    demand_dict[i].pop('Origin', None)

In [322]:
# Make LIRF the hub with a dictionary of all the airports with a 1 if it is the hub and 0 if it is not
hub = "LIRF"

g = {}
for i in airport_list:
    if i == hub:
        g[i] = 1
    else:
        g[i] = 0

In [323]:
# Revenue parameters

# Load factor
LF = 0.8

# Create Yield matrix dict from formula in RPK using distance per aircraft type using the aircraft type as the key and the rest of the columns as sub-dictionaries
# Formula: Yield = 5.9 ∙ dij^(−0.76) + 0.043

yield_matrix = {}
for k in aircraft_types:
    yield_matrix[k] = {}
    for i in airport_list:
        yield_matrix[k][i] = {}
        for j in airport_list:
            if i == j:
                yield_matrix[k][i][j] = 0
            else:
                yield_matrix[k][i][j] = 5.9 * (distance_dict[i][j] ** (-0.76)) + 0.043

yield_matrix

{'AC_1': {'LIRF': {'LIRF': 0,
   'EDDT': 0.06992760533821954,
   'RJAA': 0.048409444242827726,
   'EDDM': 0.08224233288312846,
   'LFPG': 0.07171716951350784,
   'SAEZ': 0.0479594972066314,
   'CYYZ': 0.049977078823230145,
   'MMMX': 0.048284660608056,
   'CYVR': 0.048816393354816426,
   'EDDF': 0.07491690456679825,
   'KJFK': 0.050146019628108274,
   'SBGL': 0.04875673597061972,
   'LEMD': 0.06788745977129522,
   'ZSPD': 0.04873672866660819,
   'OMDB': 0.05313252555828846,
   'LTFM': 0.06728557892472306,
   'ZBAA': 0.04927368774495451,
   'VIDP': 0.05098876368167407,
   'EFRO': 0.056847961972560174,
   'EHEH': 0.07002580717154067,
   'EFHK': 0.059783774223934535,
   'LDZA': 0.0926161307536247,
   'EBBR': 0.07038112485453549,
   'EIDW': 0.062081669675308995,
   'LGAV': 0.07205000376849223,
   'BIKF': 0.055470713046114305,
   'EETN': 0.0603273004231626,
   'LKPR': 0.07548614997834914,
   'FACT': 0.04913650249565579,
   'ESSA': 0.06109816716248087,
   'EGPH': 0.06171919391709879,
   'LFQ

In [427]:
# Cost parameters

# All aircraft are leased, and therefore a leasing cost needs to be accounted for. 
# The weekly leasing cost is a fixed amount depending on the type of aircraft

# Fuel cost formula
# CF_kij = CF_k ∙ f ∙ dij / 1.5
# Where 
# CF_kij = fuel cost for aircraft type k on route i-j [€]
# CF_k = fuel cost for aircraft type k [galon/km]
f  = 1.42 # fuel cost [€/galon]
# dij = distance between airport i and j [km]

# Time-based costs formula
# CT_kij = CT_k ∙ dij / V_k 
# Where
# CT_kij = total time-based cost for aircraft type k on route i-j [€]
# CT_k = total time-based cost for aircraft type k [€/h]
# dij = distance between airport i and j [km]
# V_k = cruise speed for aircraft type k [km/h]

# Variable costs formula
# Op_Cost_kij = CX_kij + CF_kij + CT_kij

# Fixed leg costs
# CX_k depends on the aircraft type and is a fixed cost per flight

# Create a dictionary with the Op_Cost for each aircraft type and airport using the aircraft type as the key and the rest of the columns as sub-dictionaries

Op_Cost_dict = {}
for k in aircraft_types:
    Op_Cost_dict[k] = {}
    for i in airport_list:
        Op_Cost_dict[k][i] = {}
        for j in airport_list:
            Op_Cost_dict[k][i][j] = aircraft_dict[k]['Operating_c'] + \
                                 aircraft_dict[k]['Fuel_c'] * f * distance_dict[i][j] / 1.5 + \
                                 aircraft_dict[k]['Time_c'] * distance_dict[i][j] / aircraft_dict[k]['Speed']
            # It should be noted that for flights departing or arriving at the hub airport the operating costs can be assumed to be 30% lower due to economies of scale
            if i == hub or j == hub:
                Op_Cost_dict[k][i][j] = Op_Cost_dict[k][i][j] * 0.01

In [360]:
import gurobipy as gp
from gurobipy import GRB

In [361]:
# Create model

m = gp.Model("Aircraft and network model")

In [362]:
# Create variables

# w_ij: flow from airport i to airport j that transfers at the hub
w = m.addVars(airport_list, airport_list, vtype=GRB.INTEGER, name="w") 

# x_ij: direct from airport i to airport j that does not transfer at the hub
x = m.addVars(airport_list, airport_list, vtype=GRB.INTEGER, name="x")

# z_kij: number of flights from airport i to airport j with aircraft k
z = m.addVars(aircraft_types, airport_list, airport_list, vtype=GRB.INTEGER, name="z")

# y_k: number of aircraft of type k
y = m.addVars(aircraft_types, vtype=GRB.INTEGER, name="y")
    

In [430]:
# Create objective function to maximize profit

# Revenue
# Rev1: direct revenue from direct flights (not transferring at the hub)
rev_1 = gp.quicksum((yield_matrix[k][i][j] * distance_dict[i][j] * (x[i, j] + 9 * w[i, j])) for k in aircraft_types for i in airport_list for j in airport_list)

# Rev2: revenue from connecting passengers
# Revenue generated by passengers connecting at the hub will be 10% lower
#rev_2 = 0.9 * gp.quicksum((yield_matrix[k][i][j] * w[i, j] * distance_dict[i][j]) for k in aircraft_types for i in airport_list for j in airport_list)

# Cost
# Cost1: Fixed weekly leasing cost for all aircraft types
cost_1 = gp.quicksum((aircraft_dict[k]['Lease_c'] * y[k]) for k in aircraft_types)

# Cost2: Operational costs per flight from i to j
# Op_Cost_kij = CF_kij + CT_kij + CX_k (for all aircraft types k)
# Multiply Op_Cost_kij by the corresponding z_kij to get the total cost for all flights from i to j

cost_2 = gp.quicksum((Op_Cost_dict[k][i][j] * z[k, i, j]) for k in aircraft_types for i in airport_list for j in airport_list)


# Objective function
# Full objective function with revenue and cost
m.setObjective(gp.quicksum(
    yield_matrix[k][i][j] * distance_dict[i][j] * (x[i, j] + 9 * w[i, j]) for k in aircraft_types for i in airport_list for j in airport_list) -
    gp.quicksum((aircraft_dict[k]['Lease_c'] * y[k]) for k in aircraft_types) -
    gp.quicksum((Op_Cost_dict[k][i][j] * z[k, i, j]) for k in aircraft_types for i in airport_list for j in airport_list
    ),
    GRB.MAXIMIZE)
m.update()

In [431]:
# Special hub conditions
# TAT for flights to the hub are 50% longer than the normal TAT for each aircraft type

# Add TAT to hub to the aircraft dictionary

for i in aircraft_types:
    aircraft_dict[i]['TAT_hub'] = aircraft_dict[i]['TAT'] * 0.5


In [447]:
# Add constraints

# Constraint 1: number of passengers from airport i to airport j
# x_ij + w_ij <= demand_ij (for all i and j)
m.addConstrs((x[i, j] + w[i, j] <= demand_dict[i][j] for i in airport_list for j in airport_list), name="c1")

# Constraint 2: Transfer passengers are only if the hub is not the origin or destination
# w_ij <= demand_ij * g_i * g_j (for all i and j)
m.addConstrs((w[i, j] <= demand_dict[i][j] * g[i] * g[j] for i in airport_list for j in airport_list), "c2")

# Constraint 3: capacity verification for each flight leg
# x_ij + sum(w_im * (1 - g_j) for all m) + sum(w_mj * (1 - g_i) for all m) <= sum(z_kij * s_k * LF for all k) (for all i and j)
m.addConstrs((x[i, j] + 
              gp.quicksum(w[i, m] * (1 - g[j]) for m in airport_list) + 
              gp.quicksum(w[m, j] * (1 - g[i]) for m in airport_list) <= 
              gp.quicksum(z[k, i, j] * aircraft_dict[k]['Seats'] * LF for k in aircraft_types) 
              for i in airport_list for j in airport_list), "c3")

# Constraint 4: same departing and arriving aircrafts per airport
# sum(z_kij) = sum(z_kji) (for all i and j)
m.addConstrs((gp.quicksum(z[k, i, j] for k in aircraft_types) == gp.quicksum(z[k, j, i] for k in aircraft_types) for i in airport_list for j in airport_list), "c4")


# Constraint 5: block time verification for each aircraft total
# we should add a TAT for only incoming to the hub of 50% of the normal TAT
# sum((dij / sp_k + TAT_k + (TAT_hub for all j != hub)) * z_kij for all i and j) <= BT_k * y_k (for all k)
m.addConstrs(((gp.quicksum((distance_dict[i][j] / aircraft_dict[k]['Speed'] + aircraft_dict[k]['TAT'] / 60  +
                          (aircraft_dict[k]['TAT_hub'] * (1 - g[j]) / 60)) * 
                           z[k, i, j] for i in airport_list for j in airport_list) <= 
                           aircraft_dict[k]['BT'] * y[k] * 7) for k in aircraft_types), "c5")


# Define matrix a[k, i, j] based on aircraft range and runway length
a = {}
for k in aircraft_types:
    for i in airport_list:
        for j in airport_list:
            if distance_dict[i][j] <= aircraft_dict[k]['Range'] and \
                airport_dict[i]['Runway (m)'] >= aircraft_dict[k]['Runway']:
                a[k, i, j] = 100000 
            else:
                a[k, i, j] = 0

# Constraint 6: aircraft range verification
# z_kij <= a_kij (for all k, i, j)
m.addConstrs((z[k, i, j] <= a[k, i, j] for k in aircraft_types for i in airport_list for j in airport_list), "c6")

# Constraint 7: no self flights
# z_kii = 0 (for all k and i)
m.addConstrs((z[k, i, j] == 0 for k in aircraft_types for i in airport_list for j in airport_list if i == j), "c8")


# Update model
m.update()

In [448]:
# Optimize model
m.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 46350 rows, 5448 columns and 535602 nonzeros
Model fingerprint: 0x1819e5d5
Variable types: 0 continuous, 5448 integer (0 binary)
Coefficient statistics:
  Matrix range     [6e-01, 3e+02]
  Objective range  [6e+00, 2e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 1e+05]

Loaded MIP start from previous solve with objective 1.60507e+07

Presolve removed 45541 rows and 3392 columns
Presolve time: 0.18s
Presolved: 809 rows, 2056 columns, 4942 nonzeros
Variable types: 0 continuous, 2056 integer (67 binary)

Root relaxation: objective 1.657265e+07, 183 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1.657

In [449]:
# Print the solutions as a pd.DataFrame for each aircraft type

# Total trips from i to j by aircraft type k
solution_z_k = []

for k in aircraft_types:
    solution_z = pd.DataFrame(columns=airport_list, index=airport_list)
    for v in m.getVars():
        if v.varName[0] == 'z' and v.varName[2:6] == k:
            solution_z.loc[v.varName[7:11], v.varName[12:16]] = v.x
            # Add the aircraft type to the dataframe
    print('Total flights for aircraft type: %s \n' % aircraft_dict[k]['AC_type'], solution_z, '\n')
    solution_z['Aircraft type'] = aircraft_dict[k]['AC_type']
    solution_z_k.append(solution_z)

# Make a dataframe from solution_z_k with a column for each aircraft type
solution_z_k = pd.concat(solution_z_k, axis=0)

Total flights for aircraft type: Regional Jet 
      LIRF EDDT RJAA EDDM LFPG SAEZ CYYZ MMMX CYVR EDDF  ... EIDW LGAV BIKF  \
LIRF  0.0  2.0  0.0  2.0  4.0  0.0  0.0  0.0  0.0  2.0  ...  0.0  2.0  0.0   
EDDT  2.0  0.0  0.0  2.0  0.0  0.0  0.0  0.0  0.0  2.0  ...  1.0  1.0  0.0   
RJAA  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
EDDM  2.0  2.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  2.0  ...  1.0  1.0  0.0   
LFPG  4.0  1.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  ...  0.0  0.0  1.0   
SAEZ  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
CYYZ  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
MMMX  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
CYVR  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
EDDF  2.0  2.0  0.0  2.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  1.0  0.0   
KJFK  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
SBGL  0.0  0.0  

In [458]:
# Total flow trips from i to j x_ij 
solution_x = pd.DataFrame(columns=airport_list, index=airport_list)
solution_w = pd.DataFrame(columns=airport_list, index=airport_list)

for v in m.getVars():
    if v.varName[0] == 'x':
        solution_x.loc[v.varName[2:6], v.varName[7:11]] = v.x
    elif v.varName[0] == 'w':
        solution_w.loc[v.varName[2:6], v.varName[7:11]] = v.x

print('Total pax per OD \n', solution_x, '\n')
print('Total pax per OD \n', solution_w, '\n')

Total pax per OD 
        LIRF   EDDT   RJAA   EDDM   LFPG   SAEZ CYYZ   MMMX   CYVR   EDDF  ...  \
LIRF   -0.0  112.0  155.0   91.0  181.0   63.0  0.0  102.0   35.0   71.0  ...   
EDDT   94.0   -0.0  156.0  106.0  197.0    0.0  0.0    0.0    0.0  103.0  ...   
RJAA  147.0  158.0   -0.0  128.0  256.0    0.0  0.0  256.0  139.0  119.0  ...   
EDDM   78.0  100.0  127.0   -0.0  153.0    0.0  0.0    0.0    0.0   90.0  ...   
LFPG  189.0  165.0  256.0  165.0   -0.0  127.0  0.0  189.0    0.0  168.0  ...   
SAEZ   63.0    0.0    0.0    0.0  123.0   -0.0  0.0  142.0    0.0    0.0  ...   
CYYZ    0.0    0.0    0.0    0.0    0.0    0.0 -0.0    0.0    0.0    0.0  ...   
MMMX   92.0    0.0  256.0    0.0  164.0  139.0  0.0   -0.0   80.0    0.0  ...   
CYVR   36.0    0.0  114.0    0.0    0.0    0.0  0.0   81.0   -0.0    0.0  ...   
EDDF   86.0  103.0  133.0   93.0  176.0    0.0  0.0    0.0    0.0   -0.0  ...   
KJFK  150.0  137.0  256.0  122.0  256.0  177.0  0.0  256.0  108.0  100.0  ...   
SBGL   89

In [451]:
for v in m.getVars():
    if v.varName[0] == 'w' and v.x > 0:
        print('%s %g' % (v.varName, v.x))

In [452]:
for k in aircraft_types:
    print('Total number of aircraft of type %s: %g' % (k, y[k].x))

Total number of aircraft of type AC_1: 10
Total number of aircraft of type AC_2: 10
Total number of aircraft of type AC_3: 20


In [453]:
# Export the solutions to a csv files with a long dataset format
solution_z_k.reset_index(inplace=True)
solution_z_k.rename(columns={'index': 'Origin'}, inplace=True)
solution_z_k.set_index(['Aircraft type', 'Origin'], inplace=True)

# Transform the solution_z_k dataframe to a long dataset format
solution_z_k_long = solution_z_k.stack().reset_index()
solution_z_k_long.columns = ['Aircraft type', 'Origin', 'Destination', 'Total_trips']

# Export the solution_z_long dataframe to a csv file
solution_z_k_long.to_csv('P1_Solutions/solution_trips.csv')

# Transform the solution_x_w dataframe to a long dataset format
solution_w_long = solution_w.stack().reset_index()
solution_w_long.columns = ['Origin', 'Destination', 'Total_pax']
solution_w_long['Connected_at'] = hub

# Transform the solution_x_w dataframe to a long dataset format
solution_x_long = solution_x.stack().reset_index()
solution_x_long.columns = ['Origin', 'Destination', 'Total_pax']

# # Export the solutions long dataframes to a csv file
# solution_w_long.to_csv('P1_Solutions/solution_pax_transfers.csv')
# solution_x_long.to_csv('P1_Solutions/solution_pax.csv')

In [454]:
solution_w_long

,Origin,Destination,Total_pax,Connected_at
0,LIRF,LIRF,-0.0,LIRF
1,LIRF,EDDT,0.0,LIRF
2,LIRF,RJAA,0.0,LIRF
3,LIRF,EDDM,0.0,LIRF
4,LIRF,LFPG,0.0,LIRF
...,...,...,...,...
1084,LPPT,FACT,0.0,LIRF
1085,LPPT,ESSA,0.0,LIRF
1086,LPPT,EGPH,0.0,LIRF
1087,LPPT,LFQQ,0.0,LIRF


In [455]:
# Create a OD dataframe for the airport pairs with origin and destination coordinates and total flights

airports = airport_info[['ICAO Code', 'Latitude (deg)', 'Longitude (deg)']].merge(
    airport_info[['ICAO Code', 'Latitude (deg)', 'Longitude (deg)']], how='cross', suffixes=('_origin', '_destination')
    )

# Creata a point and a line for each airport pair
airports['Origin'] = airports.apply(lambda row: Point(row['Longitude (deg)_origin'], row['Latitude (deg)_origin']), axis=1)
airports['Destination'] = airports.apply(lambda row: Point(row['Longitude (deg)_destination'], row['Latitude (deg)_destination']), axis=1)
airports['Line'] = airports.apply(lambda row: LineString([row['Origin'], row['Destination']]), axis=1)

# Add the total flights per aircraft type to the airports dataframe
for k in aircraft_types:
    airports['Total_flights_' + k] = solution_z_k_long[solution_z_k_long['Aircraft type'] == aircraft_dict[k]['AC_type']]['Total_trips'].values

# Add the total pax to the airports dataframe
airports['Total_pax'] = solution_x_w_long['Total_pax']

# Drop columns that are not needed like coordinates
airports.drop(columns=['Latitude (deg)_origin', 'Longitude (deg)_origin', 'Latitude (deg)_destination', 'Longitude (deg)_destination'], inplace=True)

# Drop the rows with same origin and destination
airports.drop(airports[airports['ICAO Code_origin'] == airports['ICAO Code_destination']].index, inplace=True)

# Create a column with the OD pair string, but if the pair is duplicated, the order is inverted
airports['OD_pair'] = np.where(airports['ICAO Code_origin'] < airports['ICAO Code_destination'], 
                               airports['ICAO Code_origin'] + '_' + airports['ICAO Code_destination'], 
                               airports['ICAO Code_destination'] + '_' + airports['ICAO Code_origin'])


# # Export the airports dataframe to a csv file
# airports.to_csv('P1_Solutions/airports.csv')

In [456]:
airports.sort_values(by='Total_pax', ascending=False)

,ICAO Code_origin,ICAO Code_destination,Origin,Destination,Line,Total_flights_AC_1,Total_flights_AC_2,Total_flights_AC_3,Total_pax,OD_pair
79,RJAA,ZSPD,POINT (140.3843 35.7702),POINT (121.8083 31.1443),"LINESTRING (140.3843 35.7702, 121.8083 31.1443)",1.0,2.0,1.0,525.0,RJAA_ZSPD
431,ZSPD,RJAA,POINT (121.8083 31.1443),POINT (140.3843 35.7702),"LINESTRING (121.8083 31.1443, 140.3843 35.7702)",1.0,3.0,0.0,424.0,RJAA_ZSPD
82,RJAA,ZBAA,POINT (140.3843 35.7702),POINT (116.6031 40.0799),"LINESTRING (140.3843 35.7702, 116.6031 40.0799)",1.0,3.0,0.0,413.0,RJAA_ZBAA
530,ZBAA,RJAA,POINT (116.6031 40.0799),POINT (140.3843 35.7702),"LINESTRING (116.6031 40.0799, 140.3843 35.7702)",1.0,3.0,0.0,396.0,RJAA_ZBAA
445,ZSPD,ZBAA,POINT (121.8083 31.1443),POINT (116.6031 40.0799),"LINESTRING (121.8083 31.1443, 116.6031 40.0799)",0.0,1.0,1.0,376.0,ZBAA_ZSPD
...,...,...,...,...,...,...,...,...,...,...
644,EHEH,VIDP,POINT (5.3784 51.4505),POINT (77.1006 28.5567),"LINESTRING (5.3784 51.4505, 77.1006 28.5567)",0.0,0.0,0.0,0.0,EHEH_VIDP
645,EHEH,EFRO,POINT (5.3784 51.4505),POINT (25.8275 66.55800000000001),"LINESTRING (5.3784 51.4505, 25.8275 66.5580000...",0.0,0.0,0.0,0.0,EFRO_EHEH
647,EHEH,EFHK,POINT (5.3784 51.4505),POINT (24.9633 60.3172),"LINESTRING (5.3784 51.4505, 24.9633 60.3172)",0.0,0.0,0.0,0.0,EFHK_EHEH
648,EHEH,LDZA,POINT (5.3784 51.4505),POINT (16.0675 45.7407),"LINESTRING (5.3784 51.4505, 16.0675 45.7407)",0.0,0.0,0.0,0.0,EHEH_LDZA


In [457]:
# # export the OP_cost_dict to a csv file and the yield matrix

# # Create a dataframe from the Op_Cost_dict
# Op_Cost_dict_df = pd.DataFrame.from_dict({(i,j): Op_Cost_dict[i][j] 
#                            for i in Op_Cost_dict.keys() 
#                            for j in Op_Cost_dict[i].keys()},
#                        orient='index')

# # Create a dataframe from the yield matrix
# yield_matrix_df = pd.DataFrame.from_dict({(i,j): yield_matrix[i][j] 
#                            for i in yield_matrix.keys() 
#                            for j in yield_matrix[i].keys()},
#                        orient='index')

# # Export the dataframes to csv files
# Op_Cost_dict_df.to_csv('Op_Cost_dict.csv')
# yield_matrix_df.to_csv('yield_matrix.csv')

